In [22]:
# Import libraries
import cv2
import numpy as np
import cPickle
import time
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import sys
import caffe
import os
#from collections import Counter

caffe.set_device(0)
caffe.set_mode_cpu()

In [23]:
def load_model_net(model_used):
    # Set the right path to your model definition file, pretrained model weights,
    # and the image you would like to classify.
    MODEL_FILE = '/home/gonzalo/models/VGG_FACE_deploy.prototxt'
    PRETRAINED = '/media/gonzalo/OS/mydata/weights_solvers/TO USE/'+model_used
    
    # load the model
    caffe.set_mode_gpu()
    caffe.set_device(0)
    net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                           #mean=np.load('/home/gonzalo/data/MIT/mean_files/7_net_mean.npy').mean(1).mean(1),
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(224, 224))
    print "successfully loaded classifier"
    return net

In [24]:
# Function: train_features()
# Description:
# Input: train_images_filenames, train_labels, extractor, n_images, isPCA=True
# Output: D, L
def train_features(images_filenames, labels, textractor, n_images, is_pca):

    # SIFTdetector = cv2.SIFT(nfeatures=n_Sift_Features)
    # read the just 30 train images per class
    # extract SIFT keypoints and descriptors
    # store descriptors in a python list of numpy arrays

    train_descriptors = []
    train_label_per_descriptor = []

    for i in range(len(images_filenames)):
        filename = images_filenames[i]
        if train_label_per_descriptor.count(labels[i]) < n_images:
            print 'Reading image ' + filename
            ima = cv2.imread(filename)
            print ima.shape
            gray = cv2.cvtColor(ima, cv2.COLOR_BGR2GRAY)
            kpt, des = textractor.detectAndCompute(gray, None)
            train_descriptors.append(des)
            train_label_per_descriptor.append(labels[i])
            print str(len(kpt)) + ' extracted keypoints and descriptors'

    d = train_descriptors[0]
    l = np.array([train_label_per_descriptor[0]] * train_descriptors[0].shape[0])

    for i in range(1, len(train_descriptors)):
        d = np.vstack((d, train_descriptors[i]))
        l = np.hstack((l, np.array([train_label_per_descriptor[i]] * train_descriptors[i].shape[0])))

    if is_pca:
        print "Apply PCA algorithm to reduce dimensionality"
        pca.fit(d)
        dtrfm = pca.transform(d)
    return dtrfm, l

In [25]:
# Function: train_SVM()
# Description:
# Input: d_scaled, l, kernel_type
# Output: clf
def train_svm(d_scaled, l, kernel_type):
    print 'Training the SVM classifier...'
    clf_train = svm.SVC(kernel=kernel_type, C=100).fit(d_scaled, l)
    print 'Done!'
    return clf_train

In [26]:
# Function: test_classifier()
# Description:
# Input: images_filenames, labels, extractor, clf, stdSlr, pca, isPCA=True
# Output: numcorrect
def classifier(images_filenames, labels, cextractor, cclf, cstdslr, is_pca, cpca,):
    numtestimages = 0
    cnumcorrect = 0
    for i in range(len(images_filenames)):
        filename = images_filenames[i]
        ima = cv2.imread(filename)
        gray = cv2.cvtColor(ima, cv2.COLOR_BGR2GRAY)

        kpt, des = cextractor.detectAndCompute(gray, None)
        if is_pca:
            dtrfm = cpca.transform(des)

        predictions = cclf.predict(cstdslr.transform(des))
        values, counts = np.unique(predictions, return_counts=True)
        predictedclass = values[np.argmax(counts)]
        print 'image ' + filename + ' was from class ' + labels[i] + ' and was predicted ' + predictedclass
        numtestimages += 1
        if predictedclass == labels[i]:
            cnumcorrect += 1
    return cnumcorrect

In [27]:
def rewrite_list(train_set_list,test_set_list):
    list_of_files, list_of_labels =[],[]
    with open(train_set_list,'r') as f, open('/home/gonzalo/data/MIT/train_0.dat','w') as g, open('/home/gonzalo/data/MIT/train_0_labels.dat','w') as h:
        for x in f:
            x = x.rstrip()
            if not x: continue
            filename, label = x.split( )
            filename = str('/home/gonzalo/data/MIT/train/'+ filename)
            list_of_files.append(filename), list_of_labels.append(label)
        cPickle.dump(list_of_files, g)
        cPickle.dump(list_of_labels, h)
            
    list_of_files, list_of_labels =[],[]        
    with open(test_set_list,'r') as i, open('/home/gonzalo/data/MIT/test_0.dat','w') as j, open('/home/gonzalo/data/MIT/test_0_labels.dat','w') as k:
        for x in i:
            x = x.rstrip()
            if not x: continue
            filename, label = x.split( )
            filename = str('/home/gonzalo/data/MIT/test/'+ filename)
            list_of_files.append(filename), list_of_labels.append(label)
        cPickle.dump(list_of_files, j)
        cPickle.dump(list_of_labels, k)

In [18]:
# ------------------ Main function ------------------ #
if __name__ == "__main__":
    
    rewrite_list('/home/gonzalo/data/MIT/train_0.txt','/home/gonzalo/data/MIT/test_0.txt')

In [28]:
    model_used = 'VGG_FACE.caffemodel'
    net = load_model_net(model_used)

    # Start timer to analyse performance
    start = time.time()

    # Read the train and test files
    train_images_filenames = cPickle.load(open('/home/gonzalo/data/MIT/train_0.dat', 'r'))
    test_images_filenames = cPickle.load(open('/home/gonzalo/data/MIT/test_0.dat', 'r'))
    
    print len(train_images_filenames), len(test_images_filenames)

successfully loaded classifier
2020 250


In [29]:
    train_labels = cPickle.load(open('/home/gonzalo/data/MIT/train_0_labels.dat', 'r'))
    test_labels = cPickle.load(open('/home/gonzalo/data/MIT/test_0_labels.dat', 'r'))

    print 'Loaded ' + str(len(train_images_filenames)) + ' training images filenames with classes ', set(train_labels)
    print 'Loaded ' + str(len(test_images_filenames)) + ' testing images filenames with classes ', set(test_labels)

    # PCA components
    pca = PCA(n_components=20)

    # Create the SIFT detector object
    extractor = net#cv2.SIFT(nfeatures=100)

    # Create the SURF detector object
    # extractor = cv2.SURF(nfeaures = 100)

    num_images = 15
    apply_pca = False
    D, L = train_features(train_images_filenames, train_labels, extractor, num_images, apply_pca)

    # Train a linear SVM classifier
    stdSlr = StandardScaler().fit(D)
    D_scaled = stdSlr.transform(D)

    kernel = 'linear'
    clf = train_svm(D_scaled, L, kernel)

    print 'Classifier trained'

    # Get all the test data and predict their labels
    numcorrect = classifier(test_images_filenames, test_labels, extractor, clf, stdSlr, True, pca)

    print 'Final accuracy: ' + str(numcorrect * 100.0 / len(test_images_filenames))

    # End timer to print time spent
    end = time.time()
    print 'Done in ' + str(end - start) + ' secs.'

Loaded 2020 training images filenames with classes  set(['0004', '0005', '0006', '0007', '0000', '0001', '0002', '0003', '0008', '0009'])
Loaded 250 testing images filenames with classes  set(['0004', '0005', '0006', '0007', '0000', '0001', '0002', '0003', '0008', '0009'])
Reading image /home/gonzalo/data/MIT/train/train_0/0000/0000_-12_0_0_15_15_1fr.jpg
(227, 227, 3)


AttributeError: 'Classifier' object has no attribute 'detectAndCompute'